In [22]:
import re
import getpass
import numpy as np
import pandas as pd
import text_input
import importlib

## Parameters

In [2]:
report  = "R1"
path2SP = f"/Users/{getpass.getuser()}/OneDrive - World Justice Project/EU Subnational/EU-S Data/reports/eu-thematic-reports"

## Reading outline

In [3]:
# Reading outline
outline = pd.read_excel(f"{path2SP}/data-viz/inputs/report_outline.xlsx")
charts4reports = (
    outline.copy()
    .loc[(outline["thematic_reports"] == True) ]
)

# Creating text input classes

In [4]:
# Reading text inputs
input_files = [
    "title-&-scroll.md",
    "introduction.md",
    # "executive-summary.md",
    "thematic-findings.md",
    "appendix.md",
    "methodology.md",
    "about.md",
    "acknowledgements.md"
]
inputs_as_classes = [text_input.text_input(report, x) for x in input_files]
inputs = dict(zip(input_files, inputs_as_classes))

## Wrangling and exporting data

In [29]:
versions = {
    "html-version" : False,
    "pdf-version"  : True
}
for version, parameter in versions.items():

    front_page = inputs["title-&-scroll.md"].get_front_page()
    intro_sections = [
        input.get_intro_sections(start_id = np.max(front_page.id)+1) 
        for type, input in inputs.items() 
        if type in ["introduction.md", "executive-summary.md"]
    ]
    thematic_findings = (
        inputs["thematic-findings.md"]
        .get_thematic_findings(
            charts4reports, 
            pdfver   = parameter, 
            id_start = np.max(intro_sections[-1].id)+1
        )
    )
    final_sections = [
        input.get_final_sections(start_id = np.max(thematic_findings.id)+1)
        for type, input in inputs.items()
        if type in ["appendix.md", "methodology.md", "about.md", "acknowledgements.md"]
    ]
    
    csv_data = pd.concat([front_page] + intro_sections + [thematic_findings] + final_sections)
    csv_data[["id", "belongs_to"]] = csv_data[["id", "belongs_to"]].astype("Int64")
    data4config = (
        csv_data.copy()
        .loc[~csv_data["id4config"].isna(), ["content", "id4config"]]
    )
    csv_data.columns = ["id", "tipo de elemento", "contenido (markdown)", "pertenece a ", "settings", "id4config"]
    (
        csv_data
        .drop(columns=["id4config"])
        .to_csv(f"{report}/{report}-csv-schema-{version}.csv", index = False, encoding = "utf-8")
    )


In [56]:
data4config["file"] = data4config["content"].apply(lambda x: re.sub(r"\.svg", ".csv", x))
data4config["id"]   = data4config["content"].apply(lambda x: re.sub(r"\.svg", "", x))

config_file = pd.merge(
    data4config,
    outline[["chart_id", "description", "type"]],
    how      = "left",
    left_on  = "id",
    right_on = "chart_id"
)

desc2names = {
    "QRQ" : "expert",
    "GPP" : "people"
}
type2names = {
    "Map"       : "map",
    "Lollipop"  : "lollypop",
    "Dumbbells" : "dumbbell"
}
config_file["description"] = config_file["description"].replace(desc2names)
config_file["type"] = config_file["type"].replace(type2names)
config_file = (
    config_file.loc[config_file["type"].isin(["map", "lollypop", "dumbbell"]), ["type", "file", "id4config", "description"]]
)
config_file.columns = ["type", "file", "id", "handler"]
config_file.to_csv(f"{report}/config.csv", index = False, encoding = "utf-8")